In [1]:
import requests
import pandas as pd
from tqdm.notebook import tqdm
import json

In [2]:
with open('token.json') as fi:
    token = json.load(fi)['token']
    
headers = {
    'Authorization': f'Bearer {token}'
}

In [3]:
org = 'nss-data-science-cohort-5'
project_name = 'hmda_shiny'

page = 1
repos = []

while page == 1 or len(response.json()) > 0:
    URL = f'https://api.github.com/orgs/{org}/repos?per_page=100&page={page}'
    response = requests.get(URL, headers = headers)
    repos.extend([x['name'] for x in response.json() if project_name in x['name']])
    page += 1

# Remove the original template repository.    
repos = [x for x in repos if x != project_name]

In [4]:
results = []

for repo_name in tqdm(repos):
    repo = f'{org}/{repo_name}'
    
    # First, retrieve all of the branches. Extract the shas from the results.
    URL = f'https://api.github.com/repos/{repo}/branches'
    response = requests.get(URL, headers = headers)
    shas = [x['commit']['sha'] for x in response.json()]

    # Then, use those shas to retrieve the details.
    
    URL = f'https://api.github.com/repos/{repo}/commits'
    for sha in shas:
        params = {
            'sha': sha,
            'per_page': 100
        }
        response = requests.get(URL, params = params, headers = headers)

        for commit in response.json():
            results.append((repo_name, 
                            commit['sha'], 
                            commit['commit']['author']['date'], 
                            commit['commit']['author']['name'],
                            commit['commit']['message']))

  0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
colnames = ['repo_name', 'sha', 'date', 'committer', 'message']

results = (
    pd.DataFrame(results, columns = colnames)
    .drop_duplicates()
)

results = results[results['committer'] != 'github-classroom[bot]']

results['date'] = pd.to_datetime(results['date'])

This can be grouped to see how many commits each individual has contributed and when the last time they committed was.

In [9]:
results.groupby(['repo_name','committer'])['date'].agg(['count', 'max'])

count  \
repo_name                     committer                  
hmda_shiny-crazy-eights       Bryan Finlayson        8   
                              MyMac                  1   
                              jrkotun                1   
                              ross-k                 6   
hmda_shiny-gum-drops          Jessica Ra             2   
                              jacob-parks98         10   
                              yvonnemartinez716      2   
hmda_shiny-loan-data          Evgeniya Kolesova     13   
                              George Mills          33   
                              Vahideh Rasekhi        9   
hmda_shiny-missile-toes       Corbec                 9   
                              James Gilbert          4   
                              ndthrkhn               1   
hmda_shiny-mr_goodbar         CReihsmann             1   
                              Conrad                 7   
                              ccmerry                6   
                              jcrando                6   
hmda_shiny-the-shiny-dancRs   Rohit Venkat           6   
                              jrioross               5   
                              rohitrvenkat           8   
                              vredd418               3   
hmda_shiny-three-musketeers-1 Chengxian Zhang       11   
                              charre2021            33   
                              czhang87              14   
                              elavender1             3   

                                                                      max  
repo_name                     committer                                    
hmda_shiny-crazy-eights       Bryan Finlayson   2021-12-17 03:23:38+00:00  
                              MyMac             2021-12-14 23:57:27+00:00  
                              jrkotun           2021-12-17 02:41:19+00:00  
                              ross-k            2021-12-22 00:04:19+00:00  
hmda_shiny-gum-drops          Jessica Ra        2021-12-17 02:10:45+00:00  
                              jacob-parks98     2021-12-22 00:37:20+00:00  
                              yvonnemartinez716 2021-12-18 19:44:03+00:00  
hmda_shiny-loan-data          Evgeniya Kolesova 2021-12-21 23:23:22+00:00  
                              George Mills      2021-12-22 01:35:41+00:00  
                              Vahideh Rasekhi   2021-12-21 17:49:44+00:00  
hmda_shiny-missile-toes       Corbec            2021-12-21 23:44:55+00:00  
                              James Gilbert     2021-12-20 01:24:22+00:00  
                              ndthrkhn          2021-12-17 03:38:29+00:00  
hmda_shiny-mr_goodbar         CReihsmann        2021-12-15 01:32:59+00:00  
                              Conrad            2021-12-21 22:31:02+00:00  
                              ccmerry           2021-12-21 23:39:45+00:00  
                              jcrando           2021-12-18 18:02:16+00:00  
hmda_shiny-the-shiny-dancRs   Rohit Venkat      2021-12-22 00:47:28+00:00  
                              jrioross          2021-12-21 19:25:28+00:00  
                              rohitrvenkat      2021-12-22 03:26:24+00:00  
                              vredd418          2021-12-21 04:29:18+00:00  
hmda_shiny-three-musketeers-1 Chengxian Zhang   2021-12-21 22:47:04+00:00  
                              charre2021        2021-12-21 19:56:45+00:00  
                              czhang87          2021-12-21 22:48:36+00:00  
                              elavender1        2021-12-19 17:53:18+00:00